# ERA5 - check input

In [ ]:
%matplotlib widget

import cartopy
import hvplot.pandas
import hvplot.xarray
import holoviews as hv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

hv.extension("bokeh")

In [ ]:
from dask.distributed import LocalCluster
cluster = LocalCluster()          # Fully-featured local Dask cluster
cluster

In [ ]:
client = cluster.get_client()
client

In [ ]:
chunks = {}
chunks = dict(
    time=120, 
    longitude=20, 
    latitude=20,
)

era5 = xr.open_mfdataset(
    "/project/home/p200232/02_meteo/era5/lon_lat/netcdf/2023*.nc",
    chunks=chunks,
)
era5

In [ ]:
window = dict(
    time=slice("2023-07-12", "2023-07-17"),
    latitude=slice(30, 0), 
    longitude=slice(210, 260),
)

era5_window = era5.sel(**window).load()

In [ ]:
# Convert MSL to kPa!
if era5_window.msl.max() > 10_000:
    era5_window["msl"] = era5_window.msl / 1000

In [ ]:
plot_options = dict(frame_width=500)

In [ ]:
msl_min = era5_window.msl.min("time")

In [ ]:
msl_min.hvplot(
    x="longitude", 
    y="latitude", 
    title="min MSL (kPa)",
    geo=True,
    coastline=True,
).options(**plot_options)

In [ ]:
# Contours make things a bit clearer
msl_min.hvplot.contourf(
    x="longitude", 
    y="latitude", 
    title="min msl (kPa)",
    levels=10,
    geo=True,
    coastline=True,
).options(**plot_options)

In [ ]:
era5_window.msl.groupby("time", squeeze=False).map(np.min).hvplot(grid=True, title="min MSL over window (kPa)")

In [ ]:
# Animation
era5_window.msl.hvplot.contourf(
    groupby="time",
    x="longitude", 
    y="latitude",
    levels=15,
    geo=True,
    coastline=True,
    widget_type="scrubber",
    widget_location="bottom",
)

In [ ]:
era5_window["mag"] = np.sqrt(era5_window.u10**2 + era5_window.v10**2)
era5_window["angle"] = (np.pi/2.) - np.arctan2(era5_window.v10, era5_window.u10)    # WTF?? I had to switch U and V in order to get the arrows to show correctly...


In [ ]:
ww = dict(latitude=slice(20, 10), longitude=slice(230, 240))
era5_window.sel(time="2023-07-14T21:00:00", **ww).hvplot.vectorfield(
    #groupby="time",
    x="longitude", 
    y="latitude",
    angle='angle', 
    mag='mag', 
    hover=False,
    geo=True,
    #widget_type="scrubber",
    #widget_location="bottom",
).opts(magnitude='mag', pivot="tip") + \
era5_window.sel(time="2023-07-14T14:00:00", **ww).hvplot.vectorfield(
    #groupby="time",
    x="longitude", 
    y="latitude",
    angle='angle', 
    mag='mag', 
    hover=False,
    geo=True,
    #widget_type="scrubber",
    #widget_location="bottom",
).opts(magnitude='mag')

In [ ]:
time_slice = slice("2023-07-10", "2023-07-17")
longitude=25
latitude=80
hv.Layout([
    era5.sel(longitude=longitude, latitude=latitude, time=time_slice).msl.hvplot(grid=True),
    era5.sel(longitude=longitude, latitude=0, time=time_slice).msl.hvplot(grid=True),
    era5.sel(longitude=longitude, latitude=-latitude, time=time_slice).msl.hvplot(grid=True),
    
]).cols(1)